In [6]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

In [7]:
# Caminho para o shapefile
shp_path = r"..\..\Data\Processed\PT-FireSprd_v2.1\L2_FireBehavior\PT-FireProg_v2.1_L2_model.shp"

# 1. Ler o shapefile
gdf = gpd.read_file(shp_path)

# 4. Listar as variáveis restantes
print("\n=== Variáveis mantidas para análise ===")
print(gdf.columns.tolist())


=== Variáveis mantidas para análise ===
['inidoy', 'enddoy', 'ros_p', 'duration_p', 'elev_av', 'aspect_av', 'landform', 'land_use', '1_3y_fir_p', '3_8y_fir_p', '8_ny_fir_p', 'fuel_model', 'f_load_av', 'sW_1m_av', 'sW_3m_av', 'sW_7_av', 'sW_28_av', 'sW_100_av', 'sW_289_av', 't_2m_C_av', 'd_2m_C_av', 'rh_2m_av', 'VPD_Pa_av', 'sP_hPa_av', 'gp_m2s2_av', 'dfmc_av', 'HDW_av', 'Haines_av', 'FWI_12h_av', 'DC_12h_av', 'FFMC_12h_a', 'wv10_kh_av', 'wdir10_av', 'wv100_k_av', 'wdir100_av', 'Recirc', 'CircVar', 't_950_av', 't_850_av', 't_700_av', 't_500_av', 't_300_av', 'rh_950_av', 'rh_850_av', 'rh_700_av', 'rh_500_av', 'rh_300_av', 'wv_950_av', 'wv_850_av', 'wv_700_av', 'wv_500_av', 'wv_300_av', 'wdi_950_av', 'wdi_850_av', 'wdi_700_av', 'wdi_500_av', 'wdi_300_av', 'vwv_950_av', 'vwv_850_av', 'vwv_700_av', 'vwv_500_av', 'vwv_300_av', 'gp_950_av', 'gp_850_av', 'gp_700_av', 'gp_500_av', 'gp_300_av', 'gT_s_9_av', 'gT_9_8_av', 'gT_8_7_av', 'gT_7_5_av', 'gT_5_3_av', 'wSv_9_av', 'wSdir_9_av', 'wSv_7_av'

In [8]:
# 3. Selecionar apenas as variáveis numéricas
num_df = gdf.select_dtypes(include=["number"])

# 4. Calcular a matriz de correlação
corr_matrix = num_df.corr()

print("\n=== Matriz de correlação (numéricas) ===")
print(corr_matrix)


=== Matriz de correlação (numéricas) ===
              inidoy    enddoy     ros_p  duration_p   elev_av  aspect_av  \
inidoy      1.000000  0.999992  0.305191    0.102364 -0.104566   0.130943   
enddoy      0.999992  1.000000  0.303759    0.106440 -0.103800   0.130921   
ros_p       0.305191  0.303759  1.000000   -0.314464 -0.178541   0.154776   
duration_p  0.102364  0.106440 -0.314464    1.000000  0.175024   0.008410   
elev_av    -0.104566 -0.103800 -0.178541    0.175024  1.000000  -0.069398   
...              ...       ...       ...         ...       ...        ...   
LiftIdx_av  0.234923  0.236288 -0.142339    0.355607 -0.186785   0.035886   
VentIdx_av  0.152361  0.151758  0.196304   -0.130413 -0.610549   0.076081   
CMLG_av    -0.075200 -0.075441  0.080964   -0.066293  0.135922   0.068488   
ros_p_lg1   0.366568  0.365779  0.628792   -0.151404 -0.222730   0.153111   
f_start    -0.220734 -0.219680 -0.187492    0.232745  0.240116  -0.039708   

            landform  land_use  1

In [ ]:
plt.figure(
    figsize=(
        max(10, len(corr_matrix.columns)),   # largura proporcional ao nº de variáveis
        max(10, 0.95 * len(corr_matrix.columns))    # altura menor para ficar mais "quadrado"
    )
)

ax = sns.heatmap(
    corr_matrix,
    cmap='coolwarm',
    center=0,
    annot=False,
    linewidths=0.3,
    cbar_kws={"shrink": 0.8}
)

plt.title(
    "Matriz de Correlação das Variáveis Ambientais e de Velocidade de Propagação do Fogo",
    fontsize=136,   # título maior
    pad=140
)

cbar = ax.collections[0].colorbar
cbar.ax.tick_params(
    labelsize=72,    # tamanho da fonte
    length=30,       # comprimento dos ticks (aumenta o espaço visual)
    width=5,        # largura dos ticks
    direction='out'  # ou 'in' para ticks para dentro
)
# Rótulos maiores e mais legíveis
plt.xticks(rotation=90, fontsize=39)
plt.yticks(rotation=0, fontsize=39)

plt.tight_layout()
plt.savefig("..\..\Data\Data_Exploration\correlation_matrix.pdf", format="pdf", dpi=300, bbox_inches="tight")
plt.show()


In [18]:
import numpy as np
import pandas as pd

n = 200
pd.set_option("display.max_rows", n)  


# Criar matriz sem diagonal
corr_mod = corr_matrix.abs().where(~np.eye(len(corr_matrix), dtype=bool))

# Manter só metade superior da matriz (remove duplicados)
corr_upper = corr_mod.where(np.triu(np.ones(corr_mod.shape), k=1).astype(bool))

# Transformar em DataFrame limpo
df_corr = (
    corr_upper
    .stack()
    .sort_values(ascending=False)
    .reset_index()
)

df_corr.columns = ["Var1", "Var2", "Correlation"]

# Pegar somente top 50
top50 = df_corr.head(n)

print(top50)


           Var1        Var2  Correlation
0        inidoy      enddoy     0.999992
1    LCL_hPa_av    LCL_m_av     0.998974
2      sW_1m_av   sW_100_av     0.996164
3      sW_3m_av   sW_289_av     0.994255
4      rh_2m_av     dfmc_av     0.990179
5     sP_hPa_av  gp_m2s2_av     0.987737
6    HigCC_p_av  TotCC_p_av     0.978762
7      sW_1m_av    sW_28_av     0.964462
8      sW_3m_av   sW_100_av     0.958627
9      rh_2m_av  LCL_hPa_av     0.954392
10     sW_1m_av    sW_3m_av     0.949760
11     rh_2m_av    LCL_m_av     0.947899
12       Recirc     CircVar     0.945576
13    t_2m_C_av   VPD_Pa_av     0.944037
14     sW_28_av   sW_100_av     0.939416
15      dfmc_av  LCL_hPa_av     0.938486
16    wv_500_av    wSv_5_av     0.933095
17    gp_500_av   gp_300_av     0.932250
18      dfmc_av    LCL_m_av     0.929894
19    sW_100_av   sW_289_av     0.923810
20    wv_700_av    wSv_7_av     0.918768
21     sW_1m_av   sW_289_av     0.910802
22    wdir10_av  wdir100_av     0.908396
23   wv100_k_av 